In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import glob
from skimage import io
import cv2
import math
from sklearn.decomposition import PCA

# First of all, we need to read images in order to train our model on this data

In [2]:
dataRoot = "DATA\\"
letters = ['ა', 'ბ', 'გ', 'დ', 'ე', 'ვ', 'ზ', 'თ', 'ი', 'კ',
                   'ლ', 'მ', 'ნ', 'ო', 'პ', 'ჟ', 'რ', 'ს', 'ტ', 'უ', 'ფ', 'ქ', 'ღ',
                   'ყ', 'შ', 'ჩ', 'ც', 'ძ', 'წ', 'ჭ', 'ხ', 'ჯ', 'ჰ']

images = []
answers = []
for i in range(33):
    img = os.path.join(dataRoot, str(i), '*')
    img_path = glob.glob(img)
    for path in img_path:
        images.append(path)
        answers.append(i)

In [3]:
trainingData = {"Image":images,"Letter": answers}
trainingData = pd.DataFrame(trainingData)
trainingData

,Image,Letter
0,DATA\0\a_!kolxeti_96.png,0
1,DATA\0\a_!kolxeti_96_15.png,0
2,DATA\0\a_!kolxeti_96_30.png,0
3,DATA\0\a_!kolxeti_bjg_96.png,0
4,DATA\0\a_!kolxeti_bjg_96_15.png,0
...,...,...
40159,DATA\32\h__kolkhety_15.png,32
40160,DATA\32\h__kolkhety_30.png,32
40161,DATA\32\h_~kolkhety_teletype.png,32
40162,DATA\32\h_~kolkhety_teletype_15.png,32


# Dataframe above contains image path in the column 'Image' and the number of letter it corresponds to in Georgian alphabet

# For example, first one is photo of Georgian 'ა', so the value of this row in column 'Letter' is 0, as 'ა' is 0-th (actually 1-st, but I've enumerated from 0) letter in Georgian alphabet

In [4]:
# This method just creates the representation array of the image where the letter is exactly in the center
def center(img):
    new_img = np.full((100, 100), 255)
    maxX = -1
    maxY = -1
    minX = 101
    minY = 101
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if img[i][j] < 255:
                maxX = max(maxX, i)
                maxY = max(maxY, j)
                minX = min(minX, i)
                minY = min(minY, j)
    letter = pd.DataFrame(img).iloc[minX:maxX+1,minY:maxY+1].to_numpy()
    width = maxX-minX+1
    height = maxY-minY+1
    moveOnX = width//2
    moveOnY = height//2
    for i in range(width):
        for j in range(height):
            new_img[50-moveOnX+i][50-moveOnY+j] = letter[i][j]
    return new_img

In [5]:
# this method gets image path as parameter, resizes the picture so that all of them has same 100x100 dimensions and 
# returns it written as np.array of pixels
def processImage(path):
    image = cv2.imread(path,0)
    image= cv2.resize(image, (100,100))
    return center(np.array(image))

In [6]:
trainingData['Pixels'] = trainingData['Image'].map(processImage)

In [7]:
# I'm going to shuffle the data randomly, in order to split them into training and test set
trainingData = trainingData.sample(frac=1).reset_index(drop=True)

In [8]:
trainingData

,Image,Letter,Pixels
0,DATA\10\l_ge-gorda_-nusx_30.png,10,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,..."
1,DATA\15\jj_geo_acadnusx_30.png,15,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,..."
2,DATA\18\t_bpg_paata_caps.png,18,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,..."
3,DATA\20\f_sggg1.png,20,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,..."
4,DATA\19\u_sanet.png,19,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,..."
...,...,...,...
40159,DATA\0\a_akademiuri_2.png,0,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,..."
40160,DATA\14\p_geodumba_bold.png,14,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,..."
40161,DATA\11\m_geo_arial_bold_15.png,11,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,..."
40162,DATA\24\ss_gf-alaverdi-mt-bold.png,24,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,..."


In [9]:
trainingSet = trainingData.iloc[0:math.trunc(trainingData.shape[0]*0.7), 0:trainingData.shape[1]]
testSet = trainingData.iloc[math.trunc(trainingData.shape[0]*0.7):trainingData.shape[0], 0: trainingData.shape[1]]

In [10]:
trainingSet

,Image,Letter,Pixels
0,DATA\10\l_ge-gorda_-nusx_30.png,10,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,..."
1,DATA\15\jj_geo_acadnusx_30.png,15,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,..."
2,DATA\18\t_bpg_paata_caps.png,18,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,..."
3,DATA\20\f_sggg1.png,20,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,..."
4,DATA\19\u_sanet.png,19,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,..."
...,...,...,...
28109,DATA\2\g_gugeshashvili_5_mthavruli.png,2,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,..."
28110,DATA\4\e_gl-lortkipanidze-bold.png,4,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,..."
28111,DATA\4\e_#dumbadze_15.png,4,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,..."
28112,DATA\10\l_geo_balaveri_regular_30.png,10,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,..."


In [11]:
# Here I just store the above mentioned arrays into the different type of array so that it will be compatible
# for the model which we will use in later parts of the assignment will be compatible 
x_train = np.ones((trainingSet.shape[0],100,100),dtype=int)
y_train = np.ones(trainingSet.shape[0],dtype=int)
x_test = np.ones((testSet.shape[0],100,100),dtype=int) 
y_test = np.ones(testSet.shape[0],dtype=int)
for i in range(trainingData.shape[0]):
    if i<trainingSet.shape[0]:
        x_train[i]=trainingData['Pixels'][i]
        y_train[i]=trainingData['Letter'][i]
    else:
        x_test[i-trainingSet.shape[0]] = trainingData['Pixels'][i]
        y_test[i-trainingSet.shape[0]] = trainingData['Letter'][i]

In [12]:
x_train

array([[[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]],

       [[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]],

       [[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]],

       ...,

       [[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255

In [13]:
x_train.shape

(28114, 100, 100)

In [14]:
y_train.shape

(28114,)

# In this project, I've used tenserflow.keras library for constructing neural network, but also here I use tf.keras.utils.normalize method to normalize the arrays of pixels

In [15]:
x_train = tf.keras.utils.normalize(x_train, axis = 1)
x_test = tf.keras.utils.normalize(x_test,axis = 1)

In [16]:
x_train

array([[[0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
        [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
        [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
        ...,
        [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
        [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
        [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1]],

       [[0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
        [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
        [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
        ...,
        [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
        [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
        [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1]],

       [[0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
        [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
        [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
        ...,
        [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
        [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
        [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1]],

       ...,

       [[0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
        [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
        [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1

In [17]:
x_test

array([[[0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
        [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
        [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
        ...,
        [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
        [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
        [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1]],

       [[0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
        [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
        [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
        ...,
        [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
        [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
        [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1]],

       [[0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
        [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
        [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
        ...,
        [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
        [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
        [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1]],

       ...,

       [[0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
        [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
        [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1

# So far, each of our picture is represented as an 100x100 array of pixels, but we need them to be one-dimensional  

In [18]:
# x_train_f will be the array of length 100000
x_train_f = np.ones((trainingSet.shape[0],10000),dtype=float)
for i in range(len(x_train)):
    x_train_f[i] = x_train[i].flatten()

In [19]:
x_train_f

array([[0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
       ...,
       [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1]])

In [20]:
# Here I do the same for test set
x_test_f = np.ones((testSet.shape[0],10000),dtype=float)
for i in range(len(x_test)):
    x_test_f[i] = x_test[i].flatten()

In [21]:
x_test_f

array([[0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
       ...,
       [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1]])

# Here we start working with neural networks. I've used tensorflow.keras implementations for them

# Firstly, just out of curiosity, I tried fitting data without using pca. I don't have high expectations, as I think 10000 features are too many for the model to learn effectively

# Also, I've tried using really simple network for starters. This is the model without any hidden layers

In [22]:
NeuralNetwork = tf.keras.models.Sequential()
NeuralNetwork.add(tf.keras.layers.Flatten())
NeuralNetwork.add(tf.keras.layers.Dense(33, activation = tf.nn.softmax))
NeuralNetwork.compile(optimizer='sgd',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [23]:
NeuralNetwork.fit(x_train_f,y_train, batch_size = 50, epochs=50)

Epoch 1/50
563/563 [==============================] - 3s 3ms/step - loss: 3.3927 - accuracy: 0.1632
Epoch 2/50
563/563 [==============================] - 1s 3ms/step - loss: 3.1963 - accuracy: 0.3432
Epoch 3/50
563/563 [==============================] - 1s 3ms/step - loss: 3.0274 - accuracy: 0.4055
Epoch 4/50
563/563 [==============================] - 1s 3ms/step - loss: 2.8829 - accuracy: 0.4420
Epoch 5/50
563/563 [==============================] - 1s 3ms/step - loss: 2.7581 - accuracy: 0.4665
Epoch 6/50
563/563 [==============================] - 1s 3ms/step - loss: 2.6511 - accuracy: 0.4814
Epoch 7/50
563/563 [==============================] - 1s 3ms/step - loss: 2.5591 - accuracy: 0.4935
Epoch 8/50
563/563 [==============================] - 1s 3ms/step - loss: 2.4788 - accuracy: 0.5058
Epoch 9/50
563/563 [==============================] - 1s 2ms/step - loss: 2.4084 - accuracy: 0.5172
Epoch 10/50
563/563 [==============================] - 1s 3ms/step - loss: 2.3466 - accuracy: 0.5262

In [24]:
(loss, acc) = NeuralNetwork.evaluate(x_test_f,y_test)
print(loss,acc)

377/377 [==============================] - 1s 911us/step - loss: 1.6140 - accuracy: 0.6671
1.6140141487121582 0.6671369075775146


# Now I'm starting to use pca to reduce the dimensions for our data

# I'm reducing data dimensions to 1000 features

In [25]:
pca = PCA(n_components = 1000)
pca.fit(x_train_f)
x_train_f = pca.transform(x_train_f)

In [26]:
x_test_f = pca.transform(x_test_f)

In [27]:
NeuralNetwork = tf.keras.models.Sequential()
NeuralNetwork.add(tf.keras.layers.Flatten())
NeuralNetwork.add(tf.keras.layers.Dense(33, activation = tf.nn.softmax))
NeuralNetwork.compile(optimizer='sgd',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [28]:
NeuralNetwork.fit(x_train_f,y_train, batch_size = 50, epochs=50)

Epoch 1/50
563/563 [==============================] - 1s 746us/step - loss: 3.3923 - accuracy: 0.1247
Epoch 2/50
563/563 [==============================] - 0s 759us/step - loss: 3.1944 - accuracy: 0.3141
Epoch 3/50
563/563 [==============================] - 1s 886us/step - loss: 3.0245 - accuracy: 0.4010
Epoch 4/50
563/563 [==============================] - 0s 795us/step - loss: 2.8790 - accuracy: 0.4385
Epoch 5/50
563/563 [==============================] - 0s 783us/step - loss: 2.7543 - accuracy: 0.4614
Epoch 6/50
563/563 [==============================] - 0s 796us/step - loss: 2.6474 - accuracy: 0.4780
Epoch 7/50
563/563 [==============================] - 0s 873us/step - loss: 2.5553 - accuracy: 0.4915
Epoch 8/50
563/563 [==============================] - 0s 833us/step - loss: 2.4754 - accuracy: 0.5035
Epoch 9/50
563/563 [==============================] - 1s 947us/step - loss: 2.4056 - accuracy: 0.5132
Epoch 10/50
563/563 [==============================] - 0s 821us/step - loss: 2.343

In [29]:
(loss, acc) = NeuralNetwork.evaluate(x_test_f,y_test)
print(loss,acc)

377/377 [==============================] - 0s 754us/step - loss: 1.6142 - accuracy: 0.6599
1.6142151355743408 0.6599169969558716


# Now I'm going to try different approaches for our model. e.g. using different activation functions, number of hidden layers, and also vary the number of units in them

# But the activation function for the last layer (activation layer) should be softmax, as our problem is multiclass, also I'm going to use sparse_categorical_crossentropy for loss function and accuracy as metric

In [30]:
NeuralNetwork = tf.keras.models.Sequential()
NeuralNetwork.add(tf.keras.layers.Flatten())
NeuralNetwork.add(tf.keras.layers.Dense(200, activation = tf.nn.relu))
NeuralNetwork.add(tf.keras.layers.Dense(33, activation = tf.nn.softmax))
NeuralNetwork.compile(optimizer='sgd',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [31]:
NeuralNetwork.fit(x_train_f,y_train, batch_size = 50, epochs=50)

Epoch 1/50
563/563 [==============================] - 1s 1ms/step - loss: 3.4335 - accuracy: 0.1018
Epoch 2/50
563/563 [==============================] - 1s 1ms/step - loss: 3.2672 - accuracy: 0.2760
Epoch 3/50
563/563 [==============================] - 1s 1ms/step - loss: 3.0545 - accuracy: 0.3833
Epoch 4/50
563/563 [==============================] - 1s 1ms/step - loss: 2.8029 - accuracy: 0.4425
Epoch 5/50
563/563 [==============================] - 1s 1ms/step - loss: 2.5511 - accuracy: 0.4838
Epoch 6/50
563/563 [==============================] - 1s 1ms/step - loss: 2.3304 - accuracy: 0.5131
Epoch 7/50
563/563 [==============================] - 1s 1ms/step - loss: 2.1492 - accuracy: 0.5380
Epoch 8/50
563/563 [==============================] - 1s 2ms/step - loss: 2.0025 - accuracy: 0.5600
Epoch 9/50
563/563 [==============================] - 1s 2ms/step - loss: 1.8825 - accuracy: 0.5778
Epoch 10/50
563/563 [==============================] - 1s 2ms/step - loss: 1.7824 - accuracy: 0.5972

In [32]:
(loss, acc) = NeuralNetwork.evaluate(x_test_f,y_test)
print(loss,acc)

377/377 [==============================] - 0s 928us/step - loss: 0.9093 - accuracy: 0.7956
0.9092722535133362 0.7956016659736633


In [33]:
NeuralNetwork = tf.keras.models.Sequential()
NeuralNetwork.add(tf.keras.layers.Flatten())
NeuralNetwork.add(tf.keras.layers.Dense(200, activation = tf.nn.sigmoid))
NeuralNetwork.add(tf.keras.layers.Dense(33, activation = tf.nn.softmax))
NeuralNetwork.compile(optimizer='sgd',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [34]:
NeuralNetwork.fit(x_train_f,y_train, batch_size = 50, epochs=50)

Epoch 1/50
563/563 [==============================] - 1s 1ms/step - loss: 3.4947 - accuracy: 0.0410
Epoch 2/50
563/563 [==============================] - 1s 1ms/step - loss: 3.4591 - accuracy: 0.0771
Epoch 3/50
563/563 [==============================] - 1s 1ms/step - loss: 3.4306 - accuracy: 0.1247
Epoch 4/50
563/563 [==============================] - 1s 1ms/step - loss: 3.4010 - accuracy: 0.1742
Epoch 5/50
563/563 [==============================] - 1s 1ms/step - loss: 3.3693 - accuracy: 0.2213
Epoch 6/50
563/563 [==============================] - 1s 1ms/step - loss: 3.3346 - accuracy: 0.2643
Epoch 7/50
563/563 [==============================] - 1s 1ms/step - loss: 3.2974 - accuracy: 0.2992
Epoch 8/50
563/563 [==============================] - 1s 1ms/step - loss: 3.2561 - accuracy: 0.3240
Epoch 9/50
563/563 [==============================] - 1s 1ms/step - loss: 3.2118 - accuracy: 0.3427
Epoch 10/50
563/563 [==============================] - 1s 1ms/step - loss: 3.1633 - accuracy: 0.3587

In [35]:
(loss, acc) = NeuralNetwork.evaluate(x_test_f,y_test)
print(loss,acc)

377/377 [==============================] - 0s 698us/step - loss: 1.6486 - accuracy: 0.6318
1.6485649347305298 0.6317842602729797


In [36]:
NeuralNetwork = tf.keras.models.Sequential()
NeuralNetwork.add(tf.keras.layers.Flatten())
NeuralNetwork.add(tf.keras.layers.Dense(500, activation = tf.nn.relu))
NeuralNetwork.add(tf.keras.layers.Dense(33, activation = tf.nn.softmax))
NeuralNetwork.compile(optimizer='sgd',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [37]:
NeuralNetwork.fit(x_train_f,y_train, batch_size = 50, epochs=50)

Epoch 1/50
563/563 [==============================] - 2s 3ms/step - loss: 3.4082 - accuracy: 0.1380
Epoch 2/50
563/563 [==============================] - 1s 3ms/step - loss: 3.1985 - accuracy: 0.3521
Epoch 3/50
563/563 [==============================] - 1s 3ms/step - loss: 2.9521 - accuracy: 0.4307
Epoch 4/50
563/563 [==============================] - 1s 2ms/step - loss: 2.6869 - accuracy: 0.4724
Epoch 5/50
563/563 [==============================] - 1s 3ms/step - loss: 2.4406 - accuracy: 0.5106
Epoch 6/50
563/563 [==============================] - 1s 3ms/step - loss: 2.2334 - accuracy: 0.5409
Epoch 7/50
563/563 [==============================] - 1s 2ms/step - loss: 2.0651 - accuracy: 0.5673
Epoch 8/50
563/563 [==============================] - 1s 2ms/step - loss: 1.9286 - accuracy: 0.5902
Epoch 9/50
563/563 [==============================] - 1s 2ms/step - loss: 1.8159 - accuracy: 0.6099
Epoch 10/50
563/563 [==============================] - 1s 2ms/step - loss: 1.7211 - accuracy: 0.6289

In [38]:
(loss, acc) = NeuralNetwork.evaluate(x_test_f,y_test)
print(loss,acc)

377/377 [==============================] - 1s 1ms/step - loss: 0.8862 - accuracy: 0.8017
0.8861677646636963 0.8017427325248718


In [39]:
NeuralNetwork = tf.keras.models.Sequential()
NeuralNetwork.add(tf.keras.layers.Flatten())
NeuralNetwork.add(tf.keras.layers.Dense(500, activation = tf.nn.sigmoid))
NeuralNetwork.add(tf.keras.layers.Dense(33, activation = tf.nn.softmax))
NeuralNetwork.compile(optimizer='sgd',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [40]:
NeuralNetwork.fit(x_train_f,y_train, batch_size = 50, epochs=50)

Epoch 1/50
563/563 [==============================] - 1s 2ms/step - loss: 3.4897 - accuracy: 0.0439
Epoch 2/50
563/563 [==============================] - 1s 2ms/step - loss: 3.4493 - accuracy: 0.0747
Epoch 3/50
563/563 [==============================] - 1s 2ms/step - loss: 3.4139 - accuracy: 0.1196
Epoch 4/50
563/563 [==============================] - 1s 2ms/step - loss: 3.3762 - accuracy: 0.1656
Epoch 5/50
563/563 [==============================] - 1s 2ms/step - loss: 3.3365 - accuracy: 0.2086
Epoch 6/50
563/563 [==============================] - 1s 2ms/step - loss: 3.2938 - accuracy: 0.2519
Epoch 7/50
563/563 [==============================] - 1s 2ms/step - loss: 3.2479 - accuracy: 0.2911
Epoch 8/50
563/563 [==============================] - 1s 2ms/step - loss: 3.1979 - accuracy: 0.3210
Epoch 9/50
563/563 [==============================] - 1s 2ms/step - loss: 3.1431 - accuracy: 0.3462
Epoch 10/50
563/563 [==============================] - 1s 2ms/step - loss: 3.0850 - accuracy: 0.3621

In [41]:
(loss, acc) = NeuralNetwork.evaluate(x_test_f,y_test)
print(loss,acc)

377/377 [==============================] - 1s 1ms/step - loss: 1.5931 - accuracy: 0.6329
1.5930582284927368 0.6329460740089417


# For now, we're starting to realize that sigmoid function doesn't really work well for our case, so from now on, I'm not going to lose time on using it

In [42]:
NeuralNetwork = tf.keras.models.Sequential()
NeuralNetwork.add(tf.keras.layers.Flatten())
NeuralNetwork.add(tf.keras.layers.Dense(200, activation = tf.nn.relu))
NeuralNetwork.add(tf.keras.layers.Dense(200, activation = tf.nn.relu))
NeuralNetwork.add(tf.keras.layers.Dense(33, activation = tf.nn.softmax))
NeuralNetwork.compile(optimizer='sgd',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [43]:
NeuralNetwork.fit(x_train_f,y_train, batch_size = 50, epochs=50)

Epoch 1/50
563/563 [==============================] - 1s 2ms/step - loss: 3.4556 - accuracy: 0.0827
Epoch 2/50
563/563 [==============================] - 1s 1ms/step - loss: 3.3416 - accuracy: 0.2302
Epoch 3/50
563/563 [==============================] - 1s 1ms/step - loss: 3.1215 - accuracy: 0.3574
Epoch 4/50
563/563 [==============================] - 1s 1ms/step - loss: 2.7502 - accuracy: 0.4303
Epoch 5/50
563/563 [==============================] - 1s 1ms/step - loss: 2.3347 - accuracy: 0.4900
Epoch 6/50
563/563 [==============================] - 1s 1ms/step - loss: 2.0021 - accuracy: 0.5407
Epoch 7/50
563/563 [==============================] - 1s 1ms/step - loss: 1.7598 - accuracy: 0.5892
Epoch 8/50
563/563 [==============================] - 1s 1ms/step - loss: 1.5824 - accuracy: 0.6248
Epoch 9/50
563/563 [==============================] - 1s 1ms/step - loss: 1.4493 - accuracy: 0.6574
Epoch 10/50
563/563 [==============================] - 1s 1ms/step - loss: 1.3464 - accuracy: 0.6833

In [44]:
(loss, acc) = NeuralNetwork.evaluate(x_test_f,y_test)
print(loss,acc)

377/377 [==============================] - 0s 879us/step - loss: 0.6265 - accuracy: 0.8518
0.6264773607254028 0.8517842292785645


In [45]:
NeuralNetwork = tf.keras.models.Sequential()
NeuralNetwork.add(tf.keras.layers.Flatten())
NeuralNetwork.add(tf.keras.layers.Dense(500, activation = tf.nn.relu))
NeuralNetwork.add(tf.keras.layers.Dense(500, activation = tf.nn.relu))
NeuralNetwork.add(tf.keras.layers.Dense(33, activation = tf.nn.softmax))
NeuralNetwork.compile(optimizer='sgd',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [46]:
NeuralNetwork.fit(x_train_f,y_train, batch_size = 50, epochs=50)

Epoch 1/50
563/563 [==============================] - 2s 4ms/step - loss: 3.4495 - accuracy: 0.1092
Epoch 2/50
563/563 [==============================] - 2s 4ms/step - loss: 3.3025 - accuracy: 0.3144
Epoch 3/50
563/563 [==============================] - 2s 4ms/step - loss: 3.0243 - accuracy: 0.4250
Epoch 4/50
563/563 [==============================] - 2s 4ms/step - loss: 2.5919 - accuracy: 0.4827
Epoch 5/50
563/563 [==============================] - 2s 4ms/step - loss: 2.1726 - accuracy: 0.5309
Epoch 6/50
563/563 [==============================] - 2s 4ms/step - loss: 1.8618 - accuracy: 0.5777
Epoch 7/50
563/563 [==============================] - 2s 4ms/step - loss: 1.6408 - accuracy: 0.6245
Epoch 8/50
563/563 [==============================] - 2s 4ms/step - loss: 1.4800 - accuracy: 0.6621
Epoch 9/50
563/563 [==============================] - 2s 4ms/step - loss: 1.3576 - accuracy: 0.6924
Epoch 10/50
563/563 [==============================] - 2s 4ms/step - loss: 1.2618 - accuracy: 0.7154

In [47]:
(loss, acc) = NeuralNetwork.evaluate(x_test_f,y_test)
print(loss,acc)

377/377 [==============================] - 1s 2ms/step - loss: 0.5978 - accuracy: 0.8575
0.5978134870529175 0.8575103878974915


In [48]:
NeuralNetwork = tf.keras.models.Sequential()
NeuralNetwork.add(tf.keras.layers.Flatten())
NeuralNetwork.add(tf.keras.layers.Dense(1500, activation = tf.nn.relu))
NeuralNetwork.add(tf.keras.layers.Dense(33, activation = tf.nn.softmax))
NeuralNetwork.compile(optimizer='sgd',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [49]:
NeuralNetwork.fit(x_train_f,y_train, batch_size = 50, epochs=50)

Epoch 1/50
563/563 [==============================] - 3s 6ms/step - loss: 3.3893 - accuracy: 0.2001
Epoch 2/50
563/563 [==============================] - 3s 5ms/step - loss: 3.1530 - accuracy: 0.4086
Epoch 3/50
563/563 [==============================] - 3s 5ms/step - loss: 2.8876 - accuracy: 0.4689
Epoch 4/50
563/563 [==============================] - 3s 5ms/step - loss: 2.6137 - accuracy: 0.5072
Epoch 5/50
563/563 [==============================] - 3s 5ms/step - loss: 2.3690 - accuracy: 0.5373
Epoch 6/50
563/563 [==============================] - 3s 5ms/step - loss: 2.1683 - accuracy: 0.5594
Epoch 7/50
563/563 [==============================] - 3s 5ms/step - loss: 2.0074 - accuracy: 0.5876
Epoch 8/50
563/563 [==============================] - 3s 5ms/step - loss: 1.8770 - accuracy: 0.6073
Epoch 9/50
563/563 [==============================] - 3s 6ms/step - loss: 1.7687 - accuracy: 0.6304
Epoch 10/50
563/563 [==============================] - 3s 6ms/step - loss: 1.6776 - accuracy: 0.6466

In [50]:
(loss, acc) = NeuralNetwork.evaluate(x_test_f,y_test)
print(loss,acc)

377/377 [==============================] - 2s 2ms/step - loss: 0.8608 - accuracy: 0.8050
0.860824704170227 0.8049792647361755


In [51]:
NeuralNetwork = tf.keras.models.Sequential()
NeuralNetwork.add(tf.keras.layers.Flatten())
NeuralNetwork.add(tf.keras.layers.Dense(2000, activation = tf.nn.relu))
NeuralNetwork.add(tf.keras.layers.Dense(33, activation = tf.nn.softmax))
NeuralNetwork.compile(optimizer='sgd',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [52]:
NeuralNetwork.fit(x_train_f,y_train, batch_size = 50, epochs=50)

Epoch 1/50
563/563 [==============================] - 4s 7ms/step - loss: 3.3924 - accuracy: 0.2012
Epoch 2/50
563/563 [==============================] - 4s 7ms/step - loss: 3.1475 - accuracy: 0.4233
Epoch 3/50
563/563 [==============================] - 4s 7ms/step - loss: 2.8767 - accuracy: 0.4766
Epoch 4/50
563/563 [==============================] - 4s 7ms/step - loss: 2.6006 - accuracy: 0.5104
Epoch 5/50
563/563 [==============================] - 4s 7ms/step - loss: 2.3554 - accuracy: 0.5413
Epoch 6/50
563/563 [==============================] - 4s 7ms/step - loss: 2.1549 - accuracy: 0.5659
Epoch 7/50
563/563 [==============================] - 4s 7ms/step - loss: 1.9945 - accuracy: 0.5917
Epoch 8/50
563/563 [==============================] - 4s 8ms/step - loss: 1.8646 - accuracy: 0.6115
Epoch 9/50
563/563 [==============================] - 4s 7ms/step - loss: 1.7572 - accuracy: 0.6326
Epoch 10/50
563/563 [==============================] - 4s 7ms/step - loss: 1.6668 - accuracy: 0.6473

In [53]:
(loss, acc) = NeuralNetwork.evaluate(x_test_f,y_test)
print(loss,acc)

377/377 [==============================] - 1s 3ms/step - loss: 0.8581 - accuracy: 0.8056
0.8580532670021057 0.8056431412696838


In [54]:
NeuralNetwork = tf.keras.models.Sequential()
NeuralNetwork.add(tf.keras.layers.Flatten())
NeuralNetwork.add(tf.keras.layers.Dense(700, activation = tf.nn.relu))
NeuralNetwork.add(tf.keras.layers.Dense(33, activation = tf.nn.softmax))
NeuralNetwork.compile(optimizer='sgd',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])


In [55]:
NeuralNetwork.fit(x_train_f,y_train, batch_size = 50, epochs=50)

Epoch 1/50
563/563 [==============================] - 2s 3ms/step - loss: 3.3975 - accuracy: 0.1516
Epoch 2/50
563/563 [==============================] - 2s 3ms/step - loss: 3.1793 - accuracy: 0.3591
Epoch 3/50
563/563 [==============================] - 2s 3ms/step - loss: 2.9272 - accuracy: 0.4366
Epoch 4/50
563/563 [==============================] - 2s 3ms/step - loss: 2.6612 - accuracy: 0.4831
Epoch 5/50
563/563 [==============================] - 2s 3ms/step - loss: 2.4181 - accuracy: 0.5147
Epoch 6/50
563/563 [==============================] - 2s 3ms/step - loss: 2.2140 - accuracy: 0.5480
Epoch 7/50
563/563 [==============================] - 2s 3ms/step - loss: 2.0483 - accuracy: 0.5717
Epoch 8/50
563/563 [==============================] - 2s 3ms/step - loss: 1.9132 - accuracy: 0.5979
Epoch 9/50
563/563 [==============================] - 2s 3ms/step - loss: 1.8012 - accuracy: 0.6164
Epoch 10/50
563/563 [==============================] - 2s 3ms/step - loss: 1.7069 - accuracy: 0.6362

In [56]:
NeuralNetwork = tf.keras.models.Sequential()
NeuralNetwork.add(tf.keras.layers.Flatten())
NeuralNetwork.add(tf.keras.layers.Dense(700, activation = tf.nn.relu))
NeuralNetwork.add(tf.keras.layers.Dense(700, activation = tf.nn.relu))
NeuralNetwork.add(tf.keras.layers.Dense(33, activation = tf.nn.softmax))
NeuralNetwork.compile(optimizer='sgd',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [57]:
NeuralNetwork.fit(x_train_f,y_train, batch_size = 50, epochs=50)

Epoch 1/50
563/563 [==============================] - 3s 6ms/step - loss: 3.4394 - accuracy: 0.1351
Epoch 2/50
563/563 [==============================] - 3s 6ms/step - loss: 3.2668 - accuracy: 0.3353
Epoch 3/50
563/563 [==============================] - 3s 6ms/step - loss: 2.9526 - accuracy: 0.4247
Epoch 4/50
563/563 [==============================] - 3s 6ms/step - loss: 2.5124 - accuracy: 0.4901
Epoch 5/50
563/563 [==============================] - 3s 6ms/step - loss: 2.1115 - accuracy: 0.5469
Epoch 6/50
563/563 [==============================] - 3s 6ms/step - loss: 1.8178 - accuracy: 0.5921
Epoch 7/50
563/563 [==============================] - 3s 5ms/step - loss: 1.6077 - accuracy: 0.6331
Epoch 8/50
563/563 [==============================] - 3s 5ms/step - loss: 1.4524 - accuracy: 0.6684
Epoch 9/50
563/563 [==============================] - 3s 5ms/step - loss: 1.3340 - accuracy: 0.6972
Epoch 10/50
563/563 [==============================] - 3s 5ms/step - loss: 1.2411 - accuracy: 0.7227

In [58]:
(loss, acc) = NeuralNetwork.evaluate(x_test_f,y_test)
print(loss,acc)

377/377 [==============================] - 1s 2ms/step - loss: 0.5838 - accuracy: 0.8608
0.5838114023208618 0.8608298897743225


In [59]:
NeuralNetwork = tf.keras.models.Sequential()
NeuralNetwork.add(tf.keras.layers.Flatten())
NeuralNetwork.add(tf.keras.layers.Dense(800, activation = tf.nn.relu))
NeuralNetwork.add(tf.keras.layers.Dense(33, activation = tf.nn.softmax))
NeuralNetwork.compile(optimizer='sgd',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [60]:
NeuralNetwork.fit(x_train_f,y_train, batch_size = 50, epochs=50)

Epoch 1/50
563/563 [==============================] - 2s 3ms/step - loss: 3.3909 - accuracy: 0.1779
Epoch 2/50
563/563 [==============================] - 2s 3ms/step - loss: 3.1656 - accuracy: 0.3751
Epoch 3/50
563/563 [==============================] - 2s 3ms/step - loss: 2.9057 - accuracy: 0.4433
Epoch 4/50
563/563 [==============================] - 2s 3ms/step - loss: 2.6322 - accuracy: 0.4856
Epoch 5/50
563/563 [==============================] - 2s 3ms/step - loss: 2.3863 - accuracy: 0.5172
Epoch 6/50
563/563 [==============================] - 2s 3ms/step - loss: 2.1851 - accuracy: 0.5460
Epoch 7/50
563/563 [==============================] - 2s 3ms/step - loss: 2.0239 - accuracy: 0.5724
Epoch 8/50
563/563 [==============================] - 2s 3ms/step - loss: 1.8934 - accuracy: 0.5959
Epoch 9/50
563/563 [==============================] - 2s 3ms/step - loss: 1.7852 - accuracy: 0.6193
Epoch 10/50
563/563 [==============================] - 2s 3ms/step - loss: 1.6939 - accuracy: 0.6365

In [61]:
(loss, acc) = NeuralNetwork.evaluate(x_test_f,y_test)
print(loss,acc)

377/377 [==============================] - 1s 2ms/step - loss: 0.8700 - accuracy: 0.8044
0.8699682950973511 0.8043983578681946


In [62]:
NeuralNetwork = tf.keras.models.Sequential()
NeuralNetwork.add(tf.keras.layers.Flatten())
NeuralNetwork.add(tf.keras.layers.Dense(200, activation = tf.nn.relu))
NeuralNetwork.add(tf.keras.layers.Dense(33, activation = tf.nn.softmax))
NeuralNetwork.compile(optimizer='adam',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [63]:
NeuralNetwork.fit(x_train_f,y_train, batch_size = 50, epochs=50)

Epoch 1/50
563/563 [==============================] - 1s 1ms/step - loss: 1.8113 - accuracy: 0.6304
Epoch 2/50
563/563 [==============================] - 1s 1ms/step - loss: 0.7619 - accuracy: 0.8268
Epoch 3/50
563/563 [==============================] - 1s 1ms/step - loss: 0.5560 - accuracy: 0.8692
Epoch 4/50
563/563 [==============================] - 1s 1ms/step - loss: 0.4387 - accuracy: 0.8986
Epoch 5/50
563/563 [==============================] - 1s 1ms/step - loss: 0.3576 - accuracy: 0.9189
Epoch 6/50
563/563 [==============================] - 1s 1ms/step - loss: 0.2948 - accuracy: 0.9353
Epoch 7/50
563/563 [==============================] - 1s 1ms/step - loss: 0.2455 - accuracy: 0.9474
Epoch 8/50
563/563 [==============================] - 1s 1ms/step - loss: 0.2066 - accuracy: 0.9559
Epoch 9/50
563/563 [==============================] - 1s 1ms/step - loss: 0.1751 - accuracy: 0.9638
Epoch 10/50
563/563 [==============================] - 1s 1ms/step - loss: 0.1499 - accuracy: 0.9693

In [64]:
(loss, acc) = NeuralNetwork.evaluate(x_test_f,y_test)
print(loss,acc)

377/377 [==============================] - 0s 772us/step - loss: 1.0311 - accuracy: 0.8740
1.031105637550354 0.8740248680114746


In [65]:
NeuralNetwork = tf.keras.models.Sequential()
NeuralNetwork.add(tf.keras.layers.Flatten())
NeuralNetwork.add(tf.keras.layers.Dense(100, activation = tf.nn.relu))
NeuralNetwork.add(tf.keras.layers.Dense(33, activation = tf.nn.softmax))
NeuralNetwork.compile(optimizer='adam',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [66]:
NeuralNetwork.fit(x_train_f,y_train, batch_size = 50, epochs=30)

Epoch 1/30
563/563 [==============================] - 1s 1ms/step - loss: 2.0690 - accuracy: 0.5680
Epoch 2/30
563/563 [==============================] - 1s 1ms/step - loss: 0.9126 - accuracy: 0.8011
Epoch 3/30
563/563 [==============================] - 1s 1ms/step - loss: 0.6792 - accuracy: 0.8435
Epoch 4/30
563/563 [==============================] - 1s 1ms/step - loss: 0.5587 - accuracy: 0.8699
Epoch 5/30
563/563 [==============================] - 1s 1ms/step - loss: 0.4759 - accuracy: 0.8881
Epoch 6/30
563/563 [==============================] - 1s 1ms/step - loss: 0.4133 - accuracy: 0.9046
Epoch 7/30
563/563 [==============================] - 1s 1ms/step - loss: 0.3605 - accuracy: 0.9194
Epoch 8/30
563/563 [==============================] - 1s 1ms/step - loss: 0.3175 - accuracy: 0.9296
Epoch 9/30
563/563 [==============================] - 1s 1ms/step - loss: 0.2812 - accuracy: 0.9386
Epoch 10/30
563/563 [==============================] - 1s 1ms/step - loss: 0.2491 - accuracy: 0.9461

In [67]:
(loss, acc) = NeuralNetwork.evaluate(x_test_f,y_test)
print(loss,acc)

377/377 [==============================] - 0s 698us/step - loss: 0.7905 - accuracy: 0.8683
0.790524423122406 0.8682987689971924


In [68]:
NeuralNetwork = tf.keras.models.Sequential()
NeuralNetwork.add(tf.keras.layers.Flatten())
NeuralNetwork.add(tf.keras.layers.Dense(50, activation = tf.nn.relu))
NeuralNetwork.add(tf.keras.layers.Dense(33, activation = tf.nn.softmax))
NeuralNetwork.compile(optimizer='adam',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [69]:
NeuralNetwork.fit(x_train_f,y_train, batch_size = 100, epochs=20)

Epoch 1/20
282/282 [==============================] - 0s 1ms/step - loss: 2.8220 - accuracy: 0.4075
Epoch 2/20
282/282 [==============================] - 0s 1ms/step - loss: 1.4793 - accuracy: 0.7134
Epoch 3/20
282/282 [==============================] - 0s 1ms/step - loss: 1.0261 - accuracy: 0.7835
Epoch 4/20
282/282 [==============================] - 0s 1ms/step - loss: 0.8377 - accuracy: 0.8150
Epoch 5/20
282/282 [==============================] - 0s 1ms/step - loss: 0.7288 - accuracy: 0.8333
Epoch 6/20
282/282 [==============================] - 0s 1ms/step - loss: 0.6537 - accuracy: 0.8484
Epoch 7/20
282/282 [==============================] - 0s 1ms/step - loss: 0.5968 - accuracy: 0.8614
Epoch 8/20
282/282 [==============================] - 0s 1ms/step - loss: 0.5503 - accuracy: 0.8718
Epoch 9/20
282/282 [==============================] - 0s 1ms/step - loss: 0.5104 - accuracy: 0.8812
Epoch 10/20
282/282 [==============================] - 0s 1ms/step - loss: 0.4771 - accuracy: 0.8889

In [70]:
(loss, acc) = NeuralNetwork.evaluate(x_test_f,y_test)
print(loss,acc)

377/377 [==============================] - 0s 684us/step - loss: 0.6594 - accuracy: 0.8452
0.6594264507293701 0.8452281951904297


# Up until now, I've tried lots of different configurations of neural networks. I've started by using 'sgd' as optimizer, than switched to 'adam', which seemed to reach high accuracy on training data faster, but the problem was overfitting every single time, the accuracy indicators for training and test sets differed by 0.1-0.15.

# So I've decided to stick to the 'sgd' function as optimizer again, and chose my final network to have one hidden layer, in which there are 500 units

In [71]:
NeuralNetwork = tf.keras.models.Sequential()
NeuralNetwork.add(tf.keras.layers.Flatten())
NeuralNetwork.add(tf.keras.layers.Dense(500, activation = tf.nn.relu))
NeuralNetwork.add(tf.keras.layers.Dense(33, activation = tf.nn.softmax))
NeuralNetwork.compile(optimizer='sgd',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [72]:
NeuralNetwork.fit(x_train_f,y_train, batch_size = 20, epochs=60)

Epoch 1/60
1406/1406 [==============================] - 3s 2ms/step - loss: 3.2485 - accuracy: 0.2778
Epoch 2/60
1406/1406 [==============================] - 2s 2ms/step - loss: 2.6292 - accuracy: 0.4879
Epoch 3/60
1406/1406 [==============================] - 2s 2ms/step - loss: 2.1046 - accuracy: 0.5562
Epoch 4/60
1406/1406 [==============================] - 2s 2ms/step - loss: 1.7883 - accuracy: 0.6138
Epoch 5/60
1406/1406 [==============================] - 2s 2ms/step - loss: 1.5853 - accuracy: 0.6530
Epoch 6/60
1406/1406 [==============================] - 3s 2ms/step - loss: 1.4417 - accuracy: 0.6828
Epoch 7/60
1406/1406 [==============================] - 3s 2ms/step - loss: 1.3334 - accuracy: 0.7094
Epoch 8/60
1406/1406 [==============================] - 3s 2ms/step - loss: 1.2480 - accuracy: 0.7305
Epoch 9/60
1406/1406 [==============================] - 2s 2ms/step - loss: 1.1788 - accuracy: 0.7461
Epoch 10/60
1406/1406 [==============================] - 2s 2ms/step - loss: 1.121

In [73]:
(loss, acc) = NeuralNetwork.evaluate(x_test_f,y_test)
print(loss,acc)

377/377 [==============================] - 1s 1ms/step - loss: 0.5967 - accuracy: 0.8595
0.5966764092445374 0.8595020771026611


In [74]:
NeuralNetwork.save('mynetwork_tsotneB')

INFO:tensorflow:Assets written to: mynetwork_tsotneB\assets


In [75]:
import pickle as pk
pk.dump(pca, open("pca_tsotneB.pkl","wb"))